## ECE 3849 Homework 5
##### Ezra Davis, Mailbox 629
##### 12/13/2016

## Problem 1:

```C
unsigned long i, m;          // r0 = i; r1 = m;
unsigned long A[10], B[10];  // r2 = &A[0]; r3 = &B[0];
B[i] = (B[i] >> 3) | (A[i] & m);
```

Assembly Code:
```as
push {r4-r6}
lsl r4, r0, #2;     r4 = r0 << 2
ldr r5, [r4, r2];   r5 = load A[i] (r4+r2)
and r5, r5, r0;     r5 = r5 & m
ldr r6, [r4, r3];   r6 = load B[i] (r4+r3)
rsr r6, r6, #3;     r6 = r6 >> 3
orr r6, r6, r5;     r6 = r6 | r5
str r6, [r4, r3];   B[i] (r4+r3) = r6
pop {r4-r6}
```

## Problem 2:

C code:
```C
long i, x;   // r0 = i; r1 = x;
long A[64];  // r2 = &A[0];

if ((i == 0) || (A[i] >= A[i - 1])) {
    x += A[i];
}
else {
    x--;
}
```
Converted Assembly:
```as
    cmp r0, #0
    bne IF
    sub r3, r0, #1             ; r3 = i-1
    ldr r4, [r2, r0, lsl #2]   ; r4 = A[i]
    ldr r3, [r2, r3, lsl #2]   ; r3 = A[i-1]
    cmp r4, r3                 ; jmp if r4 >= r3
    bge IF
    
    sub r1, r1, #1; r1 = r1 - 1 ; ELSE
    bal NEXT
IF:
    ldr r4, [r2, r0, lsl #2]   ; r4 = A[i]

NEXT:
    ; some other code
```

## Problem 3:
Original C code:
```C
long max2(long a, long b); // implemented elsewhere
long max4(long a, long b, long c, long d)
{
    return max2(max2(a, b), max2(c, d));
}
```
Assembly:
```as
max4:    ; arguments: r0=a, r1=b, r2=c, r3=d
    push {r4-r5, lr}
    mov r4, r2  ; r4 = c
    mov r5, r3  ; r5 = d
    bl max2     ;r0 = max2(a,b)
    mov r1, r5  ; r1 = d
    mov r5, r0  ; r5 = max2(a,b)
    mov r0, r4  ; r0 = c
    bl max2     ; r0 = max(c,d)
    mov r1, r0
    mov r0, r5
    bl max2
    pop {r4-r5, pc}
```


## Problem 4:

```as
;;; r2 = #8
block_copy4
        lsl   r2, r2, #2     ; 1 cycle
        mov   r3, #0         ; 1 cycle
loop1   cmp   r3, r2         ; 1 cycle
        bhs   done1          ; 1 cycle, 2 if jumping

        ldr   r12, [r1, r3]  ; 2 cycles
        str   r12, [r0, r3]  ; 2 cycles
        add   r3, r3, #4     ; 1 cycle
        b     loop1          ; 2 cycles

done1   bx lr                ; 2 cycles
```

`Result = lsl+mov+(8*loop1 to done1)+extra bhs time + bx =` $ 2 + 8*9 + 1 + 2$

block_copy4 takes 77 clock cycles to complete